In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

## Building a Spam Filter with Naive Bayes
<p>We want to build a spam filter using naive Bayes. First, we need to "teach" the computer how to classify messages. In ordert to do that, we will use the nultinomial Naive Bayes algorithm along with a dataset of 5,572 text messages, which have been already classified by humans.</p>
<p>The dataset can be found <a href = "https://archive.ics.uci.edu/ml/datasets/sms+spam+collection">here</a>.

In [2]:
spam_collection = pd.read_csv('SMSSpamCollection', 
                              sep = '\t', 
                              header = None, 
                              names = ['Label','SMS']
                             )
spam_collection.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Unfortunately, the file is too big for the DataQuest server. Hence, I will decrease the number of SMS messages here:

In [3]:
spam_collection = spam_collection.sample(n = 3000)
spam_collection.shape

(3000, 2)

The label "ham" refers to "not-spam".

In [4]:
spam_collection.isnull().sum()

Label    0
SMS      0
dtype: int64

In [5]:
spam_collection.groupby('Label').size()/spam_collection.shape[0]

Label
ham     0.862667
spam    0.137333
dtype: float64

We will split into a training and a test set: 80% of the data into the training set and 20% into the test set.

In [6]:
spam_random = spam_collection.sample(frac = 1,random_state=1)

In [7]:
index = round(spam_collection.shape[0]*0.8)
spam_train = spam_random[:index].reset_index(drop=True)
spam_test = spam_random[index:].reset_index(drop=True)

In [8]:
spam_test.head()

,Label,SMS
0,ham,Where wuld I be without my baby? The thought a...
1,spam,"Dear Voucher Holder, 2 claim this weeks offer,..."
2,ham,But you dint in touch with me.
3,ham,Hai priya are you right. What doctor said pa. ...
4,ham,K come to nordstrom when you're done


## Data Cleaning:
<p>To understand and quantify the data, we sort and count each word: a dataframe (SMS x words) in which we quantify how often each word occurs in each message.

In [9]:
re.sub('\W',' ',spam_train['SMS'].iloc[1])

'I want some cock  My hubby s away  I need a real man 2 satisfy me  Txt WIFE to 89938 for no strings action   Txt STOP 2 end  txt rec  1 50ea  OTBox 731 LA1 7WS   '

In [10]:
spam_train['SMS'] = spam_train['SMS'].apply(lambda x: re.sub('\W',' ',x)).str.lower()
spam_train['SMS'] = spam_train['SMS'].apply(lambda x: re.sub('  ',' ',x))

In [11]:
spam_train.head()

,Label,SMS
0,ham,although i told u dat i m into baig face watch...
1,spam,i want some cock my hubby s away i need a real...
2,ham,1 go to write msg 2 put on dictionary mode 3 c...
3,ham,she doesnt need any test
4,ham,okie but i scared u say i fat then u dun wan ...


Now we want to create the vocabulary: a list with all unique words used in the SMS:

In [12]:
vocabulary = []
for sms in spam_train['SMS'].str.split():
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

In [13]:
len(vocabulary)

5582

In [14]:
vocabulary

['syria',
 'madam',
 'cheat',
 'overemphasise',
 '4my',
 'freefone',
 'stop',
 'guoyang',
 'decide',
 'tcs',
 'weight',
 'ukp',
 'vary',
 'app',
 'joining',
 'askd',
 'lately',
 'lacs',
 'my',
 'opted',
 '2nite',
 'thesis',
 'tones2you',
 'mayb',
 'mrng',
 'dancce',
 'phb1',
 'rael',
 'gets',
 '45239',
 'spending',
 'quit',
 'satthen',
 '0871212025016',
 'action',
 'text',
 'university',
 'early',
 '12',
 '9t',
 '31',
 'mail',
 'yesterday',
 'o2',
 'cheap',
 '8800',
 'lyrics',
 'only',
 'basket',
 'statement',
 'devouring',
 'ummma',
 'rocks',
 'cum',
 'awkward',
 'happiness',
 'inclu',
 'selected',
 'sends',
 '2stoptxt',
 'cardiff',
 'call09050000327',
 'yelow',
 'stupid',
 'witout',
 'quiet',
 'from',
 'litres',
 '08719899229',
 'wtf',
 'bognor',
 'various',
 'patty',
 'malaria',
 'concert',
 'beforehand',
 'latest',
 'aa',
 'espe',
 'cops',
 'liver',
 'tirunelvali',
 'little',
 'cd',
 'happier',
 'studentfinancial',
 'pretsorginta',
 'energy',
 'decimal',
 'q',
 'texted',
 'ic',
 '2

We will now build a dictionary: each word corresponds to a list, which shows how many times this word appeard in the first, second, ...  last text message:

In [15]:
word_counts_per_sms = {unique_word:[0]*len(vocabulary) for unique_word in vocabulary}

In [16]:
for index, sms in enumerate(spam_train['SMS']):
    for word in sms.split():
        word_counts_per_sms[word][index] += 1

To improve the performance, we will deleted keys with only 1 entry.

In [17]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head(3)

,0,00,000,008704050406,0121,01223585236,01223585334,0125698789,02,0207,...,zealand,zed,zeros,zoom,zouk,èn,é,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The columns are sorted, therefore we see the numbers first. By cleaning columns with only 1 entry, we will change the chances of spam or ham. However, the dataframe takes too much space for the DQ server.

In [18]:
train_clean = pd.concat([spam_train,word_counts],axis = 1)
train_clean.head()

,Label,SMS,0,00,000,008704050406,0121,01223585236,01223585334,0125698789,...,zealand,zed,zeros,zoom,zouk,èn,é,ü,〨ud,鈥
0,ham,although i told u dat i m into baig face watch...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,spam,i want some cock my hubby s away i need a real...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,1 go to write msg 2 put on dictionary mode 3 c...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,she doesnt need any test,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,okie but i scared u say i fat then u dun wan ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Naive Bayes: 
<p>After cleaning the data, we can now build the spam filter. For Naive Bayes, we will need to know the probability values of these two equations:
</p>


$\large{P(Spam|w_1,w_2,...,w_n) \propto P(Spam)\dot \prod^{n} _{i=1}P(w_i|Spam)}$

$\large{P(Ham|w_1,w_2,...,w_n) \propto P(Ham)\dot \prod^{n} _{i=1}P(w_i|Ham)}$

<p>For $P(w_i|Spam)$ and $P(w_i|Ham)$, we use the additive smoothing:

$P(w_i|Spam) = \frac{N_{w_i|Spam}+\alpha}{N_{Spam}+\alpha\dot N_{Vocabulary}}$

$P(w_i|Ham) = \frac{N_{w_i|Ham}+\alpha}{N_{Ham}+\alpha\dot N_{Vocabulary}}$

<p>Some of the terms in the equations above will have the same value for every message:</p>
<ul>
<li>P(Spam) and P(Ham)</li>
<li>N<sub>Spam</sub>, N<sub>Ham</sub>, and N<sub>Vocabulary</sub></li>
</ul>

<p> We set the Laplace smoothing $\alpha=1$.

In [19]:
alpha = 1
spam = train_clean[train_clean['Label'] == 'spam'].copy()
ham = train_clean[train_clean['Label'] == 'ham'].copy()

p_spam = spam.shape[0]/train_clean.shape[0]
p_ham = ham.shape[0]/train_clean.shape[0]

In [20]:
n_spam = spam['SMS'].apply(len).sum()
n_ham = ham['SMS'].apply(len).sum()
n_voc = len(vocabulary)

In [21]:
[p_spam,p_ham,n_spam,n_ham]

[0.060193479039770695, 0.3697599426728771, 44859, 142651]

In [22]:
words_spam = {word:0 for word in vocabulary}
words_ham = {word:0 for word in vocabulary}

In [23]:
spam_train = train_clean[train_clean['Label'] == 'spam'].copy()
ham_train = train_clean[train_clean['Label'] == 'ham'].copy()

In [24]:
counter = 0
for word in vocabulary:

    n_w_spam = spam_train[word].sum()    
    p_w_spam = (n_w_spam +alpha)/(n_spam+alpha*n_voc)
    words_spam[word] = p_w_spam

    n_w_ham = ham_train[word].sum()    
    p_w_ham = (n_w_ham +alpha)/(n_ham+alpha*n_voc)
    words_ham[word] = p_w_ham

 
        

## Classifying a message
<p>We can now start to create the spam filter. It should be a function that:</p>
<ul>
<li> Takes in as input a new message $(w_1,w_2,\dots w_n$</li>
<li> Calulates $P(Spam|w_1,w_2,\dots w_n)$ and $P(Ham|w_1,w_2,\dots w_n)$</li>
<li> Compares the values $P(Spam|w_1,w_2,\dots w_n)$ and $P(Ham|w_1,w_2,\dots w_n)$ and </li>
<ul>
<li> If $P(Ham|w_1,w_2,\dots w_n)$ > $P(Spam|w_1,w_2,\dots w_n)$: classify the message as ham</li>
<li> If $P(Spam|w_1,w_2,\dots w_n)$ > $P(Ham|w_1,w_2,\dots w_n)$: classify the message as spam</li>
<li> If $P(Spam|w_1,w_2,\dots w_n)$ = $P(Ham|w_1,w_2,\dots w_n)$: undecided and we might need to think of something.</li>
</ul>
</ul>

In [25]:
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in words_spam:
            p_spam_given_message *= words_spam[word]
        if word in words_ham:
            p_ham_given_message *= words_ham[word]    
    

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [26]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 3.257402590719452e-31
P(Ham|message): 4.6195127319809316e-33
Label: Spam


In [28]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.9578706974653834e-29
P(Ham|message): 2.0537815380623668e-25
Label: Ham


Now we test the spam filter on the test set. First, we need to rewrite the classify function to use it on the test data set:

In [31]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in words_spam:
            p_spam_given_message *= words_spam[word]

        if word in words_ham:
            p_ham_given_message *= words_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [32]:
spam_test['predicted'] = spam_test['SMS'].apply(classify_test_set)
spam_test.head()

,Label,SMS,predicted
0,ham,Where wuld I be without my baby? The thought a...,ham
1,spam,"Dear Voucher Holder, 2 claim this weeks offer,...",spam
2,ham,But you dint in touch with me.,ham
3,ham,Hai priya are you right. What doctor said pa. ...,ham
4,ham,K come to nordstrom when you're done,ham


We will now look at the accuracy:

In [34]:
(spam_test['Label'] == spam_test['predicted']).sum()/spam_test.shape[0]

0.9833333333333333

This is quite a good accuracy. Next, we'll look at the messages that have been categorised incorrectly:

In [35]:
spam_test[spam_test['Label'] != spam_test['predicted']]

,Label,SMS,predicted
157,spam,LookAtMe!: Thanks for your purchase of a video...,ham
202,spam,LIFE has never been this much fun and great un...,ham
263,spam,"Hi babe its Jordan, how r u? Im home from abro...",ham
264,ham,Waiting for your call.,spam
275,ham,We have sent JD for Customer Service cum Accou...,spam
280,spam,"Hi babe its Chloe, how r u? I was smashed on s...",ham
330,spam,TBS/PERSOLVO. been chasing us since Sept for£3...,ham
359,spam,"Did you hear about the new ""Divorce Barbie""? I...",ham
494,ham,For me the love should start with attraction.i...,needs human classification
516,ham,Gettin rdy to ship comp,spam


The messages that got categorised incorrectly, do not display a "typical" spam jargon, which makes it difficult for the algorithm to classify.

## Conclusion
<p>We build a spam filter using Naive Bayes. We used this <a href = "https://archive.ics.uci.edu/ml/datasets/sms+spam+collection">data set</a> containing numerous SMS messages, which have been already classified into spam and ham. Splitting the data set into a test and train set, we cleaned the train set and computed the probabilities of each word to appear in a spam or ham message. Using these probabilities, we were able to classify the SMS messages from the test set with an accuracy of 